In [429]:
# %run elon_functions.py
import pandas as pd
import re
import datetime as dt
from tqdm import tqdm
import pandas as pd
import numpy as np
import os
import datetime
import mysql.connector
from pandas.io import sql
import sqlalchemy
import re
from baseball_scraper import playerid_lookup
# Give the location of the file 
loc = ("data/GameLogs_1419.csv") 
# To open Workbook 
df = pd.read_csv(loc)


print(len(df))
to_delete = []
#delete the double headers. Confuses the model.
for row, col in df.iterrows():
    r = df.iloc[row]
    if (len(r["Date"]) >10):
        to_delete.append(row)
df = df.drop(to_delete, axis = 0)      
print("deleted double header rows")
df.reset_index(inplace = True, drop = True)

170705
deleted double header rows


In [430]:
df["Date"] = [x[0:10] for x in df["Date"]]
df['Date'] = df['Date'].apply(lambda x: dt.datetime.strptime(x,'%Y-%m-%d'))

In [132]:
def make_player_id_dict(df):
    player_names = list(df["Player"].unique())
    name_id_dict = {}
    for x in player_names:
        first_name = re.findall('(.*) ', x)[0]
        last_name = re.findall('.* (.*)\\\\', x)[0]
        
        if (first_name[1] =='.'):
            first_name = first_name[:2] + ' ' + first_name[2:]
        lookup_id = 0
        try:
            lookup_id_df = playerid_lookup(last_name, first_name)
            if(len(lookup_id_df) > 1):
                lookup_id_df = lookup_id_df.dropna()
                if(len(lookup_id_df) >1):
                    lookup_id = -1 # player has two people with the same name
                else:
                    lookup_id = lookup_id_df['key_mlbam'].values[0]
            else:
                lookup_id = lookup_id_df['key_mlbam'].values[0]
        except:
            lookup_id = -1
        if (lookup_id != -1):
            name_id_dict[(first_name, last_name)] = lookup_id
    
    return name_id_dict

my_dict = make_player_id_dict(df)

In [482]:
def append_next_game_details(df2):
    #determine if the player gets a hit the next game
    df_huge = pd.DataFrame()
    player_names= list(df2["Name"].unique())
    for name in player_names:
        df_player = df2[df2["Name"] == name]
        df_p = df_player.sort_values(by = ["fixed_date"])
        df_p["next_game_BA"] = df_p["BA"].shift(-1)
        df_p["next_game_Name"] = df_p["Name"].shift(-1)
        df_p["next_game_Date"] = df_p["fixed_date"].shift(-1)
        df_p["key"] = df_p["key"].shift(-1)
        df_huge = pd.concat([df_huge, df_p], axis = 0)
    return df_huge    

In [55]:
#dumping the contents of my_dict into a picklized file
import pickle
with open('data/mydict.pickle','wb') as handle:
    pickle.dump(my_dict, handle)

In [433]:
list(my_dict)[-1]

('Tim', 'Locastro')

# Once the scraping is complete, we then combine our two dataframes

In [434]:
loc = ("data/aggregates.csv") 
# To open Workbook 
df2 = pd.read_csv(loc)

In [435]:
dates = []
for x in df2["Game_date"]:
    try:
        j = dt.datetime.strptime(x,'%m/%d/%Y')
    except:
        try:
            j = dt.datetime.strptime(x,'%Y-%m-%d %H:%M:%S')
        except:
            try:
                j = dt.datetime.strptime(x,'%m/%d/%Y %H:%M')
            except:
                print(x, "didn't pass third case")
    dates.append(j)

In [436]:
assert(len(dates) == len(df2))
df2["fixed_date"] = [str(k)[:10] for k in dates]

Create a hash key with the player name and date information in order to merge df and df2

In [437]:
name_date = []
#make the keys for the game log df
for row,col in df.iterrows():
    row = df.iloc[row]
    player_name = row["Player"]
    first_name = re.findall('(.*) ', player_name)[0]
    #if the name has a '.' in it like "A.J."
    if (first_name[1] =='.'):
        first_name = first_name[:2] + ' ' + first_name[2:]
    last_name = re.findall('.* (.*)\\\\', player_name)[0]
    name = first_name + ' ' + last_name
    date = str(row["Date"])[:10]
    name_date.append(hash(name+date))
df["key"] = name_date

In [438]:
name_date = []
#make the key for the aggregate stats df
for row,col in df2.iterrows():
    row = df2.iloc[row]
    player_name = row["Name"]
    date = row["fixed_date"]
#     print(player_name+date)
    name_date.append(hash(player_name+date))
#create a key that will enable us to merge dataframes. 
df2["key"] = name_date

In [483]:
df2 = append_next_game_details(df2)

In [602]:
# dic = {}
# duplicates = []
# for j in df["key"]:
#     if (j in dic):
#         duplicates.append(j)
#     else:
#         dic[j] = 1
# print(duplicates)

In [517]:
#joins are only possible if the keys and their datatypes are the same
df2["key"] =df2["key"].astype(float)
df["key"] = df["key"].astype(float)

In [591]:
mega_df = df2.join(df.set_index('key'), on='key', rsuffix='agg')

In [592]:
mega_df[mega_df["Pos.Summary"].notnull()]

,BA,BABIP,BIP,Game_date,Games_played_to_date,ISO,LA_avg,LA_median,Name,OBP,...,X3B,HR,RBI,BB,SO,WPA,RE24,aLI,BOP,Pos.Summary
2,0.285714,0.400000,0.714286,5/6/2014,3,0.428571,NaN,NaN,Steven Tolleson,0.375000,...,1.0,0.0,0.0,0.0,0.0,0.082,1.570,0.452,9.0,2B
5,0.428571,0.545455,0.785714,5/10/2014,6,0.571429,NaN,NaN,Steven Tolleson,0.500000,...,0.0,0.0,0.0,0.0,2.0,-0.052,-1.155,0.508,9.0,2B
9,0.318182,0.437500,0.695652,5/16/2014,10,0.333992,NaN,NaN,Steven Tolleson,0.384615,...,0.0,0.0,0.0,1.0,1.0,-0.083,-0.843,0.934,7.0,2B
10,0.269231,0.368421,0.703704,5/17/2014,11,0.286325,NaN,NaN,Steven Tolleson,0.354839,...,0.0,0.0,0.0,0.0,2.0,-0.088,-0.976,0.635,7.0,2B
12,0.258065,0.347826,0.696970,5/21/2014,13,0.226784,NaN,NaN,Steven Tolleson,0.324324,...,0.0,0.0,0.0,0.0,2.0,0.025,0.301,0.315,7.0,2B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267279,0.252427,0.308370,0.753205,2014-08-24 00:00:00,96,0.138598,NaN,NaN,Ryan Ludwick,0.310145,...,0.0,0.0,0.0,0.0,2.0,-0.073,-0.648,0.720,5.0,LF
267282,0.249211,0.306034,0.750000,2014-08-31 00:00:00,99,0.135164,NaN,NaN,Ryan Ludwick,0.307910,...,0.0,0.0,0.0,0.0,1.0,0.013,0.311,0.245,6.0,DH
267288,0.252252,0.311203,0.741840,2014-09-13 00:00:00,105,0.136472,NaN,NaN,Ryan Ludwick,0.309140,...,0.0,0.0,1.0,1.0,0.0,0.075,1.385,0.690,7.0,LF
267289,0.255952,0.315574,0.744118,2014-09-14 00:00:00,106,0.135224,NaN,NaN,Ryan Ludwick,0.313830,...,0.0,0.0,0.0,0.0,0.0,-0.174,-1.203,1.665,4.0,LF


In [593]:
bad_cols = ['160421_183314_percentage_hits', '160421_204540_percentage_hits',
       '160421_194503_percentage_hits', '160421_182611_percentage_hits',
       '160421_195036_percentage_hits', '160421_191133_percentage_hits',
       '160421_200605_percentage_hits', '160421_202513_percentage_hits',
       '160421_193312_percentage_hits', '160421_195422_percentage_hits',
       '160421_182835_percentage_hits','160421_182316_percentage_hits', '160421_185447_percentage_hits']

In [594]:
mega_df = mega_df.drop(columns = ["Unnamed: 0", "Game_date","fixed_date"] + bad_cols)

In [595]:
mega_df.columns

Index(['BA', 'BABIP', 'BIP', 'Games_played_to_date', 'ISO', 'LA_avg',
       'LA_median', 'Name', 'OBP', 'OPS', 'PA', 'SLG', 'Walks', 'mlbam_code',
       'FT_percentage_hits', 'pHitsByZone5', 'Weak_lsa_p', 'Topped_lsa_p',
       'Under_lsa_p', 'Flare/Burner_lsa_p', 'SolidContact_lsa_p',
       'Barrel_lsa_p', 'FF_percentage_hits', 'FC_percentage_hits',
       'SI_percentage_hits', 'pHitsByZone2', 'pHitsByZone4', 'pHitsByZone14',
       'CH_percentage_hits', 'SL_percentage_hits', 'pHitsByZone1',
       'pHitsByZone13', 'pHitsByZone6', 'KC_percentage_hits', 'pHitsByZone7',
       'pHitsByZone9', 'CU_percentage_hits', 'FS_percentage_hits',
       'pHitsByZone8', 'pHitsByZone12', 'pHitsByZone3', 'pHitsByZone11',
       'FO_percentage_hits', 'KN_percentage_hits', 'FA_percentage_hits',
       'EP_percentage_hits', 'SC_percentage_hits', 'key', 'next_game_BA',
       'next_game_Name', 'next_game_Date', 'next_game_Key', 'Player', 'Date',
       'Tm', 'Opp', 'PAagg', 'AB', 'R', 'H', 'X2B', 'X3B

In [596]:
mega_df = mega_df.reindex(columns=['Name', 'Tm', 'BA', 'BABIP', 'BIP', 'Games_played_to_date', 'ISO', 'LA_avg', 'LA_median','OBP', 'OPS', 'PA', 'SLG', 'Walks', 'mlbam_code',
       'FT_percentage_hits', 'pHitsByZone5', 'Weak_lsa_p', 'Topped_lsa_p',
       'Under_lsa_p', 'Flare/Burner_lsa_p', 'SolidContact_lsa_p',
       'Barrel_lsa_p', 'FF_percentage_hits', 'FC_percentage_hits',
       'SI_percentage_hits', 'pHitsByZone2', 'pHitsByZone4', 'pHitsByZone14',
       'CH_percentage_hits', 'SL_percentage_hits', 'pHitsByZone1',
       'pHitsByZone13', 'pHitsByZone6', 'KC_percentage_hits', 'pHitsByZone7',
       'pHitsByZone9', 'CU_percentage_hits', 'FS_percentage_hits',
       'pHitsByZone8', 'pHitsByZone12', 'pHitsByZone3', 'pHitsByZone11',
       'FO_percentage_hits', 'KN_percentage_hits', 'FA_percentage_hits',
       'EP_percentage_hits', 'SC_percentage_hits', 'key', 'next_game_BA',
       'next_game_Name', 'next_game_Date', 'next_game_Key', 'Player', 'Date','Opp', 'PAagg', 'AB', 'R', 'H', 'X2B', 'X3B', 'HR', 'RBI', 'BB',
       'SO', 'WPA', 'RE24', 'aLI', 'BOP', 'Pos.Summary'])

In [597]:
for col in mega_df.columns:
    if (mega_df[col].dtype =='float64'):
        mega_df[col] = mega_df[col].round(5)

In [598]:
mega_df = mega_df[mega_df['BA'].notna()]

In [599]:
mega_df= mega_df.reset_index()

In [584]:
mega_df.to_csv('data/aggregate_stats_v2.csv')

In [600]:
database_username = 'admin'
database_password = 'd8ascience!'
database_ip       = ''
database_name     = ''

database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name), pool_timeout = 28800, max_overflow = -1)
connection = database_connection.connect()
print("made connection")
# try:
num = 0
try:
    while(num < (len(mega_df))):
        ie = 'append'
        if (num == 0):
            ie = 'replace'
        val = min((len(mega_df) -num), 6000)
        print(num,val)
        mega_df[num: num+ val].to_sql('aggregate_stats_v1', connection,index=True, if_exists=ie, chunksize = 25000)
        print("table "+ ie + " successful")
        num +=val
except:
    print("error")
    connection.close()
connection.close()

made connection
0 6000
table replace successful
6000 6000
table append successful
12000 6000
table append successful
18000 6000
table append successful
24000 6000
table append successful
30000 6000
table append successful
36000 6000
table append successful
42000 6000
table append successful
48000 6000
table append successful
54000 6000
table append successful
60000 6000


Exception during reset or similar
Traceback (most recent call last):
  File "c:\users\elong\anaconda3\envs\dsu\lib\site-packages\sqlalchemy\pool\base.py", line 693, in _finalize_fairy
    fairy._reset(pool)
  File "c:\users\elong\anaconda3\envs\dsu\lib\site-packages\sqlalchemy\pool\base.py", line 880, in _reset
    pool._dialect.do_rollback(self)
  File "c:\users\elong\anaconda3\envs\dsu\lib\site-packages\sqlalchemy\dialects\mysql\base.py", line 2337, in do_rollback
    dbapi_connection.rollback()
  File "c:\users\elong\anaconda3\envs\dsu\lib\site-packages\mysql\connector\connection_cext.py", line 386, in rollback
    self._cmysql.rollback()
_mysql_connector.MySQLInterfaceError: MySQL server has gone away


table append successful
66000 6000
table append successful
72000 6000
table append successful
78000 6000
table append successful
84000 6000
table append successful
90000 6000
table append successful
96000 6000
table append successful
102000 6000
table append successful
108000 6000
table append successful
114000 6000
table append successful
120000 6000
table append successful
126000 6000
table append successful
132000 6000
table append successful
138000 6000
table append successful
144000 6000
table append successful
150000 6000
table append successful
156000 6000
table append successful
162000 6000
table append successful
168000 6000
table append successful
174000 6000
table append successful
180000 6000
table append successful
186000 6000
table append successful
192000 6000
table append successful
198000 6000
table append successful
204000 6000
table append successful
210000 6000
table append successful
216000 6000
table append successful
222000 6000
table append successful
228000 600

In [337]:
import pickle
my_dict ={}
with open('data/mydict.pickle','rb') as handle:
    my_dict = pickle.dumps(handle)